# Installation

The following instructions were prepared using

In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9900X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake-avx512)
Environment:
  JULIA_NUM_THREADS = 4


Before exploring the notebook you need to clone the main repository:
```bash
 git clone https://github.com/kalmarek/1812.03456.git
```
This notebook should be located in `1812.03456/notebooks` directory.

In the main directory (`1812.03456`) you should run the following code in `julia`s `REPL` console to instantiate the environment for computations:
```julia
using Pkg
Pkg.activate(".")
Pkg.instantiate()
```
(this needs to be done once per installation).

Instantiation should install (among others): the [`SCS` solver][1], [`JuMP` package][2] for mathematical programming and `IntervalArithmetic.jl` package from [`ValidatedNumerics.jl`][3].

The environment uses [`Groups.jl`][7], [`GroupRings.jl`][6] (which are built on the framework of  [`AbstractAlgebra.jl`][4]) and [`PropertyT.jl`][8] packages.

[1]: https://github.com/cvxgrp/scs  
[2]: https://github.com/JuliaOpt/JuMP.jl  
[3]: https://github.com/JuliaIntervals/ValidatedNumerics.jl
[4]: https://github.com/Nemocas/AbstractAlgebra.jl
[5]: https://github.com/Nemocas/Nemo.jl
[6]: https://github.com/kalmarek/GroupRings.jl
[7]: https://github.com/kalmarek/Groups.jl
[8]: https://github.com/kalmarek/PropertyT.jl

A jupyter server may be launched then within the directory `1812.03456` by issuing from julia command-line (`REPL`) the following commands.
```julia
using Pkg
Pkg.activate(".")
using IJulia
notebook(dir=".")

```
During the first run the user may be asked for installation of `Jupyter` program (a server for running this notebook) within `miniconda` environment, which will happen automatically after confirmation. To execute the commands in the notebook, one needs to navigate to `notebooks` subdirectory of `1812.03456` and click either of the notebooks.

> Code "cells" have a label, e.g. `In [1]:` on their left margin. Click somewhere in the proximity of the label to select a cell. The selected cell is _run_ by either clicking the `Run` button in the top toolbar, or by pressing `Shift`+`Enter`.

# The computation

The following notebook uses the precomputed solutions to certify that

$$\operatorname{Adj}_5 + 2\operatorname{Op}_5 - 0.28\Delta_5 \simeq \Sigma_i \xi_i^*\xi_i \in \Sigma^2_2 I\operatorname{SAut}(F_5),$$

and that

$$\operatorname{Adj}_5 + 3\operatorname{Op}_5 - 1.4 \Delta_5 \simeq \Sigma_i \xi_i^*\xi_i \in \Sigma^2_2 I\operatorname{SAut}(F_5),$$

where $\simeq \Sigma_i \xi_i^*\xi_i$ means "admits approximate sum of squares decomposition" in the language of [arXiv:1812.03456](https://arxiv.org/abs/1812.03456).

These are the computations upon the main results of [arXiv:1812.03456](https://arxiv.org/abs/1812.03456) depend.
Additionaly, one may execute the whole program (including the optimization) proving that

$$\operatorname{Adj}_4 + 100 \mathrm{Op}_4 - 0.1\Delta_4 \simeq \Sigma_i \xi_i^*\xi_i \in \Sigma^2_2 I\operatorname{SAut}(F_4).$$

This result shows that almost all groups $\operatorname{SAut}(F_n)$ have property (T). While the constants in the result are by no means optimal, is should be achievable fairly quickly (at most an hour) on a standard laptop computer.

In [2]:
using Pkg
Pkg.activate("..")
using Dates
now()

 Activating environment at `~/1812.03456/Project.toml`


2020-10-21T13:39:54.894

In [3]:
using SparseArrays
import LinearAlgebra: norm
using AbstractAlgebra

using Groups
using GroupRings
using PropertyT

using PropertyT.JLD

using PropertyT.IntervalArithmetic
IntervalArithmetic.setrounding(Interval, :tight)
IntervalArithmetic.setformat(:standard, sigfigs=6);

So far we only made the needed packages available in the notebook. You need to dowload `SAutF5_r2.tar.xz` file from zenodo.org (`doi:10.5281/zenodo.4118043`) and unpack it in the `1812.03456` directory. In the next cell we define the directory where the precomputed solutions should reside and check that required files are in place.

In [4]:
const N = 5
const prefix = abspath(joinpath(@__DIR__,  ".." , "SAutF$(N)_r2")) 
isdir(prefix) || @error "You need to place the precomputed solutions downloaded from zenodo in \n $prefix"

const DELTA_FILE = joinpath(prefix,"delta.jld");
const SQADJOP_FILE = joinpath(prefix, "SqAdjOp_coeffs.jld");
const BLOCKDEC_FILE = joinpath(prefix, "BlockDecomposition.jld");

if !(isfile(DELTA_FILE) && isfile(SQADJOP_FILE) && isfile(BLOCKDEC_FILE))
    @error "The directory structure must be preserved exactly as in the archive downloaded from zenodo:
    $prefix \n\t $DELTA_FILE \n\t $SQADJOP_FILE \n\t $ORBITDATA_FILE
    must exist! 
    "
end

Below we define a simple function which takes a group ring element `eoi` and a matrix `Q` and computes the the sum of squares `sos` group ring element which `Q` represents (in the parent group ring of `eoi`) and returns the difference `eoi - sos`. This function will be used to compare target elements with the ones obtained through semi-definite optimization.

In [5]:
function SOS_residual(eoi::GroupRingElem, Q::AbstractMatrix)
    RG = parent(eoi)
    @time sos = PropertyT.compute_SOS(RG, Q);
    return eoi - sos
end

SOS_residual (generic function with 1 method)

## Loading the precomputed group ring elements
Below we load $\Delta$, the group Laplacian for $\operatorname{SAut}(F_5)$, vectors of coefficients defining elements $\operatorname{Sq}_5$, $\operatorname{Adj}_5$ and $\operatorname{Op}_5$ and turn these into data structures `GroupRingElem` which represent group ring elements.

Note that the vectors of coefficients only have meaning with respect to the particular basis which was used for computations. The basis is not saved and loaded due to space concerns. It can be re-computed using
```julia
let basis, sizes = Groups.generate_balls(gens(G), one(G), radius=4)
    @assert sizes[2] == size(RG.pm, 1) == size(RG.pm, 2)
    RG.basis, RG.basis_dict = basis, GroupRings.reverse_dict(basis)
end
```

In [6]:
G = SAut(FreeGroup(N))
Δ = PropertyT.loadGRElem(DELTA_FILE, G)
RG = parent(Δ)
block_decomposition = let od = load(BLOCKDEC_FILE, "BlockDecomposition")
    block_decomposition = PropertyT.decimate(od, false) # sparsification of projection matrices
end
Sq, Adj, Op = let (sq_c, adj_c, op_c) = load(SQADJOP_FILE, "Sq", "Adj", "Op")
    sq = GroupRingElem(sq_c, RG)
    adj = GroupRingElem(adj_c, RG)
    op = GroupRingElem(op_c, RG)
    sq, adj, op
end;

## Replicating $\operatorname{Adj}_5 + 2\operatorname{Op}_5 - 0.28\Delta_5 \in \Sigma^2_2 ISAut(F_5)$

Let us define: $\lambda = 0.28$ and $k=2$ and check that precomputed solution file is in place:

In [7]:
const λ = 0.28
const k = 2.0

const fullpath = joinpath(prefix, "$(λ)_K=$k")
const SOLUTION_FILE = joinpath(fullpath, "solution.jld")

if !(isdir(fullpath) && isfile(SOLUTION_FILE))
    @error "Couldn't find the precomputed solution: \n\t$SOLUTION_FILE"
end

Let us finally define the element of interest (we don't show it by default by placing `;` at the end of line):

In [8]:
elt = Adj + k*Op;

`Q` loaded below is the matrix whose columns represent the elements $\xi_i \in \mathbb{R}\operatorname{SAut}(F_5)$ such that
$$
\operatorname{Adj}_5 + 2\operatorname{Op}_5 - 0.28 \Delta_5 - \sum_i \xi_i^* \xi_i = b,
$$
where $\|b\|_1$ is sufficiently small with respect to $\lambda$ (i.e. `Q` represents _approximate sum of squares_ in the language of [arXiv:1812.03456](https://arxiv.org/abs/1812.03456).

In [9]:
Q = load(SOLUTION_FILE, "Q")

4641×4641 Array{Float64,2}:
  2.97397     -0.244942     -0.244942     …   0.0158543     0.0131965
 -0.244942     7.72331      -0.311095         0.000722162  -0.000870519
 -0.244942    -0.311095      7.72331         -0.00516345   -0.000870519
 -0.244942    -0.311095     -0.311095        -0.00615118   -0.00498285
 -0.244942    -0.311095     -0.311095         0.000245145  -0.00112319
 -0.244942    -0.00261826   -0.136534     …   0.000722162  -0.000870519
 -0.244942    -0.136534     -0.161489        -0.00516345   -0.000870519
 -0.244942    -0.136534      0.0399134       -0.00615118   -0.00498285
 -0.244942    -0.136534      0.0399134        0.000245145  -0.00112319
 -0.244942    -0.136534     -0.00261826      -0.00153488   -0.000870519
 -0.244942    -0.161489     -0.136534     …  -0.00153488   -0.000870519
 -0.244942     0.0399134    -0.136534        -0.0101051    -0.00498285
 -0.244942     0.0399134    -0.136534        -0.00738246   -0.00112319
  ⋮                                       ⋱ 

Below we compute the residual $b$. As we do it in floating-point arithmetic, the result can't be taken too seriously.

In [10]:
let EOI = elt - λ*Δ
    residual = SOS_residual(EOI, Q)
    @info "In floating point arithmetic the ℓ₁-norm of the residual" norm(residual, 1);
end

 28.313046 seconds (245.26 k allocations: 270.780 MiB)


┌ Info: In floating point arithmetic the ℓ₁-norm of the residual
│   norm(residual, 1) = 9.173410298989073e-5
└ @ Main In[10]:3


Therefore we resort to interval arithmetic to provide certified upper and lower bounds on the norm of the residual:
* We first change entries of `Q` to narrow intervals,
* We project columns of `Q` so that $0$ is in the sum of coefficients of each column (i.e. $\xi_i \in I \operatorname{SAut}(F_5)$),
* We compute the sum of squares and the $\ell_1$-norm of the residual in the interval arithmetic.

The returned `check_columns_augmentation` is a boolean flag to detect if the projection was successful, i.e. if we can guarantee that each column of `Q_aug` can be represented by an element from the augmentation ideal. (If it were not successful, one may project `Q = PropertyT.augIdproj(Q)` in the floating point arithmetic prior to the cell below).

In [11]:
Q_aug, check_columns_augmentation = PropertyT.augIdproj(Interval, Q);
@assert check_columns_augmentation
Q_aug

4641×4641 Array{Interval{Float64},2}:
  [2.97397, 2.97398]         …   [0.0131964, 0.0131965]
 [-0.244942, -0.244941]         [-0.000870514, -0.000870513]
 [-0.244942, -0.244941]         [-0.000870514, -0.000870513]
 [-0.244942, -0.244941]         [-0.00498285, -0.00498284]
 [-0.244942, -0.244941]         [-0.00112319, -0.00112318]
 [-0.244942, -0.244941]      …  [-0.000870514, -0.000870513]
 [-0.244942, -0.244941]         [-0.000870514, -0.000870513]
 [-0.244942, -0.244941]         [-0.00498285, -0.00498284]
 [-0.244942, -0.244941]         [-0.00112319, -0.00112318]
 [-0.244942, -0.244941]         [-0.000870514, -0.000870513]
 [-0.244942, -0.244941]      …  [-0.000870514, -0.000870513]
 [-0.244942, -0.244941]         [-0.00498285, -0.00498284]
 [-0.244942, -0.244941]         [-0.00112319, -0.00112318]
   ⋮                         ⋱    ⋮
 [-0.00908475, -0.00908474]      [0.00250892, 0.00250893]
 [-0.0125012, -0.0125011]    …  [-0.00325656, -0.00325655]
 [-0.0125012, -0.0125011]       [

We now create the element of interest, $\operatorname{Adj}_5 + 2\operatorname{Op}_5 - 0.28 \Delta_5$ with coefficients in real intervals. Note that only $\lambda$ is turned into a non-exact interval, and except the support of $\Delta$ (the first `49` elements) all coefficients are honest integers.

In [12]:
elt_int = elt - @interval(λ)*Δ;
elt_int.coeffs

11154301-element SparseVector{Interval{Float64},Int64} with 4161 stored entries:
  [1       ]  =  [7657.59, 7657.61]
  [2       ]  =  [-191.721, -191.719]
  [3       ]  =  [-191.721, -191.719]
  [4       ]  =  [-191.721, -191.719]
  [5       ]  =  [-191.721, -191.719]
  [6       ]  =  [-191.721, -191.719]
  [7       ]  =  [-191.721, -191.719]
  [8       ]  =  [-191.721, -191.719]
  [9       ]  =  [-191.721, -191.719]
  [10      ]  =  [-191.721, -191.719]
              ⋮
  [4628    ]  =  [1, 1]
  [4629    ]  =  [1, 1]
  [4631    ]  =  [1, 1]
  [4632    ]  =  [1, 1]
  [4633    ]  =  [1, 1]
  [4634    ]  =  [1, 1]
  [4635    ]  =  [1, 1]
  [4636    ]  =  [1, 1]
  [4638    ]  =  [1, 1]
  [4639    ]  =  [1, 1]
  [4640    ]  =  [1, 1]

Finally we compute the $\ell_1$ norm of the residual with interval arithmetic. The result is **guaranteed** to be contained in the resulting interval, i.e. if each entry of `Q` were changed into an honest rational number and all the computations were carried out in rational arithmetic, the rational $\ell_1$-norm will be contained in the interval $\ell_1$-norm.

In [13]:
residual_int = SOS_residual(elt_int, Q_aug)
@show norm(residual_int, 1);

953.889613 seconds (520.88 k allocations: 529.781 MiB, 0.00% gc time)
norm(residual_int, 1) = [0.000228394, 0.000228496]


Using Lemma 4.10 of [arXiv:1812.03456](https://arxiv.org/abs/1812.03456) and the fact that none of the generating transvections of $\operatorname{SAut}(F_N)$ is involutive we obtain

In [14]:
λ_cert = @interval(λ) - 2*norm(residual_int,1)
@info "λ is certified to be > " λ_cert.lo
@info "i.e Adj_$N + $k·Op_$N - ($(λ_cert.lo))·Δ_$N ∈ Σ²₂ ISAut(F_$N)"

┌ Info: λ is certified to be > 
│   λ_cert.lo = 0.2795430082668905
└ @ Main In[14]:2
┌ Info: i.e Adj_5 + 2.0·Op_5 - (0.2795430082668905)·Δ_5 ∈ Σ²₂ ISAut(F_5)
└ @ Main In[14]:3


## Replicating $\operatorname{Adj}_5 + 3\operatorname{Op}_5 - 1.4\Delta_5 \in \Sigma^2_2 ISAut(F_5)$

In [15]:
const λ = 1.4
const k = 3.0

const fullpath = joinpath(prefix, "$(λ)_K=$k")
const SOLUTION_FILE = joinpath(fullpath, "solution.jld")

if !(isdir(fullpath) && isfile(SOLUTION_FILE))
    @error "Couldn't find the precomputed solution: \n\t$SOLUTION_FILE"
end

In [16]:
elt = Adj + k*Op;
Q = load(SOLUTION_FILE, "Q");
Q_aug, check_columns_augmentation = PropertyT.augIdproj(Interval, Q);
@assert check_columns_augmentation

In [17]:
elt_int = elt - @interval(λ)*Δ;
residual_int = SOS_residual(elt_int, Q_aug)
@show norm(residual_int, 1);

953.420002 seconds (113.01 k allocations: 512.971 MiB, 0.05% gc time)
norm(residual_int, 1) = [0.0149432, 0.0149435]


In [18]:
λ_cert = @interval(λ) - 2*norm(residual_int,1)
@info "λ is certified to be > " λ_cert.lo
@info "i.e Adj_$N + $k·Op_$N - ($(λ_cert.lo))·Δ_$N ∈ Σ²₂ ISAut(F_$N)"

┌ Info: λ is certified to be > 
│   λ_cert.lo = 1.3701131734136716
└ @ Main In[18]:2
┌ Info: i.e Adj_5 + 3.0·Op_5 - (1.3701131734136716)·Δ_5 ∈ Σ²₂ ISAut(F_5)
└ @ Main In[18]:3


## Confirming that $\operatorname{Adj}_4 + 100.0\operatorname{Op}_4 - 0.1\Delta_4$ admits approximate sum of squares decomposition

In [19]:
const λ = 0.1
const k = 100.0

100.0

Now we will generate the ball $B_R$ of radius $R=4$ in $\operatorname{SAut}(F_4)$, and use it as a (partial) basis of a group ring. Such a group ring also needs a (partial) multiplication table to square elements supported on radius $2$ (it is called `pm`, which is actually a "division table" due to `twisted=true` argument). The table is created as follows: when $x$,$y$ reside at positions `i` and `j` in $B_R$, then `pm[i,j] = k`, where `k` is the position of $x^{-1}y$ in $B_R$.

Then we create the Laplace operator $\Delta$.

In [20]:
SAutF₄ = SAut(FreeGroup(4))
Δ₄ = let G = SAutF₄, halfradius = 2
    
    S = gens(G)
    S = [S; inv.(S)] # generating set must be symmetric
    
    B_R, sizes = Groups.wlmetric_ball(S, radius=2*halfradius);
    B_rdict = GroupRings.reverse_dict(B_R)
    pm = GroupRings.create_pm(B_R, B_rdict, sizes[halfradius]; twisted=true);
    RG = GroupRing(G, B_R, B_rdict, pm)
    @show sizes;
    Δ = length(S)*one(RG) - sum(RG(s) for s in S)
end

sizes = [49, 1777, 57725, 1777541]


48(id) - 1ϱ₁₂ - 1ϱ₁₃ - 1ϱ₁₄ - 1ϱ₂₁ - 1ϱ₂₃ - 1ϱ₂₄ - 1ϱ₃₁ - 1ϱ₃₂ - 1ϱ₃₄ - 1ϱ₄₁ - 1ϱ₄₂ - 1ϱ₄₃ - 1λ₁₂ - 1λ₁₃ - 1λ₁₄ - 1λ₂₁ - 1λ₂₃ - 1λ₂₄ - 1λ₃₁ - 1λ₃₂ - 1λ₃₄ - 1λ₄₁ - 1λ₄₂ - 1λ₄₃ - 1ϱ₁₂^-1 - 1ϱ₁₃^-1 - 1ϱ₁₄^-1 - 1ϱ₂₁^-1 - 1ϱ₂₃^-1 - 1ϱ₂₄^-1 - 1ϱ₃₁^-1 - 1ϱ₃₂^-1 - 1ϱ₃₄^-1 - 1ϱ₄₁^-1 - 1ϱ₄₂^-1 - 1ϱ₄₃^-1 - 1λ₁₂^-1 - 1λ₁₃^-1 - 1λ₁₄^-1 - 1λ₂₁^-1 - 1λ₂₃^-1 - 1λ₂₄^-1 - 1λ₃₁^-1 - 1λ₃₂^-1 - 1λ₃₄^-1 - 1λ₄₁^-1 - 1λ₄₂^-1 - 1λ₄₃^-1

In the cell below we compute Wedderburn-Artin decomposition of `pm` with respect to the action of the Weyl group ($S_2 \wr S_4$) and sparsify it numerically.

In [21]:
block_decomposition = let od = PropertyT.BlockDecomposition(parent(Δ₄), WreathProduct(SymmetricGroup(2), SymmetricGroup(4)))
    PropertyT.decimate(od, false);
end;

┌ Info: Decomposing basis of RG into orbits of
│   autS = Wreath Product of Full symmetric group over 2 elements by Full symmetric group over 4 elements
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:15


 48.762163 seconds (1.11 G allocations: 76.049 GiB, 37.93% gc time)


┌ Info: The action has 6231 orbits
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:18
┌ Info: Finding projections in the Group Ring of
│   autS = Wreath Product of Full symmetric group over 2 elements by Full symmetric group over 4 elements
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:20


  4.558152 seconds (6.03 M allocations: 308.531 MiB, 15.56% gc time)


┌ Info: Finding AutS-action matrix representation
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:23


 11.694259 seconds (291.32 M allocations: 19.976 GiB, 34.16% gc time)
  0.392333 seconds (1.13 M allocations: 120.585 MiB)


┌ Info: Computing the projection matrices Uπs
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:27


 31.218765 seconds (1.29 M allocations: 8.241 GiB, 2.36% gc time)


┌ Info: 
│ multiplicities  =   6  27  38  23  18  22  56  34  22  23  27  32   9  17   8   1   1   1   2   1
│     dimensions  =   1   3   3   2   1   4   8   4   6   6   6   6   4   8   4   1   3   3   2   1
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/blockdecomposition.jl:37


In [22]:
@time Adj₄ = PropertyT.Adj(parent(Δ₄), 4)
@time Op₄ = PropertyT.Op(parent(Δ₄), 4)

elt₄ = Adj₄ + k*Op₄

  7.023643 seconds (1.36 M allocations: 71.502 MiB)
  1.688224 seconds (190.03 k allocations: 10.537 MiB)


39936(id) - 1664ϱ₁₂ - 1664ϱ₁₃ - 1664ϱ₁₄ - 1664ϱ₂₁ - 1664ϱ₂₃ - 1664ϱ₂₄ - 1664ϱ₃₁ - 1664ϱ₃₂ - 1664ϱ₃₄ - 1664ϱ₄₁ - 1664ϱ₄₂ - 1664ϱ₄₃ - 1664λ₁₂ - 1664λ₁₃ - 1664λ₁₄ - 1664λ₂₁ - 1664λ₂₃ - 1664λ₂₄ - 1664λ₃₁ - 1664λ₃₂ - 1664λ₃₄ - 1664λ₄₁ - 1664λ₄₂ - 1664λ₄₃ - 1664ϱ₁₂^-1 - 1664ϱ₁₃^-1 - 1664ϱ₁₄^-1 - 1664ϱ₂₁^-1 - 1664ϱ₂₃^-1 - 1664ϱ₂₄^-1 - 1664ϱ₃₁^-1 - 1664ϱ₃₂^-1 - 1664ϱ₃₄^-1 - 1664ϱ₄₁^-1 - 1664ϱ₄₂^-1 - 1664ϱ₄₃^-1 - 1664λ₁₂^-1 - 1664λ₁₃^-1 - 1664λ₁₄^-1 - 1664λ₂₁^-1 - 1664λ₂₃^-1 - 1664λ₂₄^-1 - 1664λ₃₁^-1 - 1664λ₃₂^-1 - 1664λ₃₄^-1 - 1664λ₄₁^-1 - 1664λ₄₂^-1 - 1664λ₄₃^-1 + ϱ₁₂*ϱ₁₃ + ϱ₁₂*ϱ₁₄ + ϱ₁₂*ϱ₂₃ + ϱ₁₂*ϱ₂₄ + ϱ₁₂*ϱ₃₁ + 2ϱ₁₂*ϱ₃₂ + 200ϱ₁₂*ϱ₃₄ + ϱ₁₂*ϱ₄₁ + 2ϱ₁₂*ϱ₄₂ + 200ϱ₁₂*ϱ₄₃ + 2ϱ₁₂*λ₁₃ + 2ϱ₁₂*λ₁₄ + ϱ₁₂*λ₂₃ + ϱ₁₂*λ₂₄ + ϱ₁₂*λ₃₁ + 2ϱ₁₂*λ₃₂ + 200ϱ₁₂*λ₃₄ + ϱ₁₂*λ₄₁ + 2ϱ₁₂*λ₄₂ + 200ϱ₁₂*λ₄₃ + ϱ₁₂*ϱ₁₃^-1 + ϱ₁₂*ϱ₁₄^-1 + ϱ₁₂*ϱ₂₃^-1 + ϱ₁₂*ϱ₂₄^-1 + ϱ₁₂*ϱ₃₁^-1 + 2ϱ₁₂*ϱ₃₂^-1 + 200ϱ₁₂*ϱ₃₄^-1 + ϱ₁₂*ϱ₄₁^-1 + 2ϱ₁₂*ϱ₄₂^-1 + 200ϱ₁₂*ϱ₄₃^-1 + 2ϱ₁₂*λ₁₃^-1 + 2ϱ₁₂*λ₁₄^-1 + ϱ₁₂*λ₂₃^-1 + ϱ₁₂*λ₂₄^-1 + ϱ₁₂*λ₃₁^-1 + 2ϱ₁₂*λ₃₂^-1 +

We are ready to define the optimisation problem. Function
> `PropertyT.SOS_problem(x, Δ, orbit_data; upper_bound=UB)`  

defines the optimisation problem equivalent to the one of the form
\begin{align}
\text{ maximize : } \quad & \lambda\\
\text{under constraints : }\quad & 0 \leqslant \lambda \leqslant \operatorname{UB},\\
     & x - \lambda \Delta = \sum \xi_i^* \xi_i,\\
     & \text{with $\sum \xi_i^* \xi_i$ invariant under the action of $S_2 \wr S_4$}.
\end{align}

In [23]:
SDP_problem, varP = PropertyT.SOS_problem_primal(elt₄, Δ₄, block_decomposition; upper_bound=0.1);
(SDP_problem,)

┌ Info: Adding 6231 constraints...
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/vcGsE/src/sos_sdps.jl:124


 24.120746 seconds (8.20 M allocations: 33.581 GiB, 7.96% gc time)


(A JuMP Model
Maximization problem with:
Variables: 11047
Objective function type: JuMP.VariableRef
`JuMP.GenericAffExpr{Float64,JuMP.VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6231 constraints
`Array{JuMP.GenericAffExpr{Float64,JuMP.VariableRef},1}`-in-`MathOptInterface.PositiveSemidefiniteConeSquare`: 20 constraints
`JuMP.VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: λ,)

### Solving the problem
Depending on the actual problem one may need to tweak the parameters given to the solver:
 * `eps` sets the requested accuracy,
 * `max_iters` sets the number of iterations to run before solver gives up,
 * `alpha` is a bias parameter ($\alpha \in (0,2)$) which determines the step size (hence the rate of convergence). $\alpha$ close to $2$ shoudl provide fastest initial convergence, but may introduce numerical noise.
 * `acceleration_lookback`: if you experience numerical instability in scs log should be changed to `0`.
 
The parameters below should be enough to obtain a decent solution for $\operatorname{Adj}_4 + 100.0\operatorname{Op}_4 - 0.1\Delta_4$ in reasonable time (<10 minutes).

In [24]:
using PropertyT.JuMP
using SCS
warm = nothing

In [25]:
with_SCS = with_optimizer(SCS.Optimizer, 
    linear_solver=SCS.DirectSolver, 
    eps=1e-10,
    max_iters=10_000,
    alpha=1.5,
    acceleration_lookback=25,
    warm_start=true)

status, warm = PropertyT.solve(SDP_problem, with_SCS, warm);
@show status;

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 8010504
eps = 1.00e-10, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
acceleration_lookback = 25, rho_x = 1.00e-03
Variables n = 11047, constraints m = 17278
Cones:	primal zero / dual free vars: 11570
	linear vars: 1
	sd vars: 5707, sd blks: 20
Setup time: 8.97e+01s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.46e+20  2.60e+20  1.00e+00 -2.31e+23  1.87e+21  5.59e+22  1.99e-01 
   100| 8.05e-04  4.66e-04  5.00e-02  4.02e-02 -1.24e-02  1.95e-15  1.05e+01 
   200| 3.06e-04  2.0

NOTE: if `status = ALMOST_OPTIMAL`, you may need to re-run the cell above.

It seems that solver obtained sum of squares decomposition for $\operatorname{Adj}_4 + 100.0\operatorname{Op}_4 - 0.1\Delta_4$ with $\lambda$:

In [26]:
λ₄ = value(SDP_problem[:λ])

0.09999999998782166

To apply the `SOS_residual` function we need to reconstruct matrix `Q` from the semipositive-definite blocks in `SDP_problem`: 

In [27]:
Q₄ = let Ps = [value.(P) for P in varP], bd = block_decomposition
    Qs = real.(sqrt.(Ps));
    PropertyT.reconstruct(Qs, block_decomposition);
end

1777×1777 Array{Float64,2}:
 73.6668      -3.53685      -3.53685      …   0.042403      0.052693
 -3.53685     20.088        -0.672753        -0.00720344   -0.0118432
 -3.53685     -0.672753     20.088           -0.0412075     4.17635e-5
 -3.53685     -0.672753     -0.672753         0.0136568     0.00637814
 -3.53685     -0.909577     -0.399332         0.00507484   -0.0118432
 -3.53685     -0.399332     -0.431527     …  -0.000939495   4.17635e-5
 -3.53685     -0.399332      1.35135          0.000623127   0.00637814
 -3.53685     -0.399332     -0.909577         0.00937818   -0.000377992
 -3.53685     -0.431527     -0.399332         0.00280739   -0.000377992
 -3.53685      1.35135      -0.399332         0.00367772    0.00719231
 -3.53685     -0.399332     -0.399332     …   0.0318183     0.00811301
 -3.53685     -0.431527      1.35135         -0.00728109    0.00811301
 -3.53685      1.35135      -0.431527         0.00512306    0.00785316
  ⋮                                       ⋱   ⋮    

In [28]:
Q₄_aug, check_columns_augmentation = PropertyT.augIdproj(Interval, Q₄);
@assert check_columns_augmentation

In [29]:
elt₄_int = elt₄ - @interval(λ₄)*Δ₄;
residual₄_int = SOS_residual(elt₄_int, Q₄_aug)
@show norm(residual₄_int, 1);
λ₄_cert = @interval(λ₄) - 2*norm(residual₄_int, 1)
@info "λ₄ is certified to be > " λ₄_cert.lo
@info "i.e Adj₄ + $k·Op₄ - ($(λ₄_cert.lo))·Δ₄ ∈ Σ²₂ ISAut(F₄)"

 54.242839 seconds (40.43 k allocations: 80.632 MiB)
norm(residual₄_int, 1) = [1.08665e-05, 1.09478e-05]


┌ Info: λ₄ is certified to be > 
│   λ₄_cert.lo = 0.09997810453347668
└ @ Main In[29]:5
┌ Info: i.e Adj₄ + 100.0·Op₄ - (0.09997810453347668)·Δ₄ ∈ Σ²₂ ISAut(F₄)
└ @ Main In[29]:6


In [30]:
now()

2020-10-21T14:24:45.043